In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader


In [11]:
X,Y= load_boston(return_X_y=True)

In [12]:
x_train,x_test,y_train,y_test=train_test_split(X,Y)

In [13]:
def normalize(x):
  mean=x.mean(axis=0)
  std=x.std(axis=0)
  return (x-mean/std)
X,Y= load_boston(return_X_y=True)
print(X[0])
X=normalize(X)
print(X[0])


[6.320e-03 1.800e+01 2.310e+00 0.000e+00 5.380e-01 6.575e+00 6.520e+01
 4.090e+00 1.000e+00 2.960e+02 1.530e+01 3.969e+02 4.980e+00]
[-4.14197417e-01  1.75122776e+01  6.85039912e-01 -2.72598567e-01
 -4.25363901e+00 -2.37845891e+00  6.27614373e+01  2.28595383e+00
 -9.78032094e-02  2.93575364e+02  6.76685348e+00  3.92989298e+02
  3.20637225e+00]


In [ ]:
y_train=y_train.reshape(379,1)

In [14]:
import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset

class bostonData(Dataset):
    def __init__(self):
        X,Y= load_boston(return_X_y=True)
        print(X[0])
        X=self.normalize(X)
        x_train,x_test,y_train,y_test=train_test_split(X,Y)
        self.x,self.y = x_train,y_train
        print(X[0])
    def normalize(self,x):
      mean=x.mean(axis=0)
      std=x.std(axis=0)
      return (x-mean/std)
    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]


In [ ]:
next(iter(x))

[tensor([[1.3262e-01, 0.0000e+00, 8.5600e+00, 0.0000e+00, 5.2000e-01, 5.8510e+00,
          9.6700e+01, 2.1069e+00, 5.0000e+00, 3.8400e+02, 2.0900e+01, 3.9405e+02,
          1.6470e+01],
         [2.2876e-01, 0.0000e+00, 8.5600e+00, 0.0000e+00, 5.2000e-01, 6.4050e+00,
          8.5400e+01, 2.7147e+00, 5.0000e+00, 3.8400e+02, 2.0900e+01, 7.0800e+01,
          1.0630e+01],
         [1.6566e+00, 0.0000e+00, 1.9580e+01, 0.0000e+00, 8.7100e-01, 6.1220e+00,
          9.7300e+01, 1.6180e+00, 5.0000e+00, 4.0300e+02, 1.4700e+01, 3.7280e+02,
          1.4100e+01],
         [1.6760e-01, 0.0000e+00, 7.3800e+00, 0.0000e+00, 4.9300e-01, 6.4260e+00,
          5.2300e+01, 4.5404e+00, 5.0000e+00, 2.8700e+02, 1.9600e+01, 3.9690e+02,
          7.2000e+00],
         [1.1432e-01, 0.0000e+00, 8.5600e+00, 0.0000e+00, 5.2000e-01, 6.7810e+00,
          7.1300e+01, 2.8561e+00, 5.0000e+00, 3.8400e+02, 2.0900e+01, 3.9558e+02,
          7.6700e+00]], dtype=torch.float64),
 tensor([19.5000, 18.6000, 21.5000, 23.800

In [ ]:
x_train.shape[1]

13

XW+b

In [28]:

class LinearRegression():
  def __init__(self):
    self.w=1
    self.b=0
      
  def train(self,batch,lr,num_epocs):
    dataset=DataLoader(bostonData(), batch_size=batch, shuffle=True)
    self.w=torch.randn(x_train.shape[1],1, requires_grad=True,dtype=torch.double)
    self.b=torch.randn(1,requires_grad=True,dtype=torch.double)
    #SGD
    for epoc in range(num_epocs):
      x,y=next(iter(dataset))
      z=self.obFunc(x)
      loss=self.loss()
      loss(z,y).sum().backward()
      self.updateParams([self.w,self.b],lr=lr,batch=batch)
      with torch.no_grad():
        train_loss=self.loss()(z,y)
        print(f'epoch {epoc + 1}, loss {float(train_loss.mean()):f}')
  def updateParams(self,params,lr,batch):
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad / batch
            param.grad.zero_()
  def loss(self):
    return torch.nn.MSELoss()
  def obFunc(self,x):
    return torch.matmul(x,self.w)+self.b
  def predict(self,x):
    return torch.matmul(torch.tensor(x,requires_grad=False),self.w)+self.b
  def test(self,x_test,y_test):
    import numpy as np
    return np.sqrt((self.predict(x_test).detach().numpy()-y_test.reshape(127,1))**2).sum()/127


In [29]:
model=LinearRegression()
model.train(batch=35,lr=0.0001,num_epocs=5000)

[6.320e-03 1.800e+01 2.310e+00 0.000e+00 5.380e-01 6.575e+00 6.520e+01
 4.090e+00 1.000e+00 2.960e+02 1.530e+01 3.969e+02 4.980e+00]
[-4.14197417e-01  1.75122776e+01  6.85039912e-01 -2.72598567e-01
 -4.25363901e+00 -2.37845891e+00  6.27614373e+01  2.28595383e+00
 -9.78032094e-02  2.93575364e+02  6.76685348e+00  3.92989298e+02
  3.20637225e+00]
epoch 1, loss 321611.209192
epoch 2, loss 254077.516869
epoch 3, loss 131112.126423
epoch 4, loss 87984.243391
epoch 5, loss 56692.926934
epoch 6, loss 27687.189369
epoch 7, loss 13254.060245
epoch 8, loss 10545.754285
epoch 9, loss 6659.338145
epoch 10, loss 6185.502989
epoch 11, loss 4735.208863
epoch 12, loss 5599.756458
epoch 13, loss 5114.487510
epoch 14, loss 4919.989881
epoch 15, loss 4544.458900
epoch 16, loss 3430.136388
epoch 17, loss 5326.826436
epoch 18, loss 3688.539546
epoch 19, loss 3317.574075
epoch 20, loss 3411.699171
epoch 21, loss 3936.563479
epoch 22, loss 4772.320277
epoch 23, loss 3953.415317
epoch 24, loss 5489.499569
epoc

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([35])) that is different to the input size (torch.Size([35, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch 124, loss 1639.604771
epoch 125, loss 2409.315394
epoch 126, loss 2737.462177
epoch 127, loss 2603.384969
epoch 128, loss 1722.802145
epoch 129, loss 1978.591713
epoch 130, loss 1738.991511
epoch 131, loss 1569.892574
epoch 132, loss 2269.230987
epoch 133, loss 1880.080954
epoch 134, loss 2606.722774
epoch 135, loss 2950.356245
epoch 136, loss 1771.428506
epoch 137, loss 2037.340033
epoch 138, loss 1507.678332
epoch 139, loss 1913.745417
epoch 140, loss 1730.506174
epoch 141, loss 1946.939845
epoch 142, loss 1984.117085
epoch 143, loss 2081.341642
epoch 144, loss 1184.235529
epoch 145, loss 2279.227878
epoch 146, loss 1585.789235
epoch 147, loss 1860.429750
epoch 148, loss 2193.083042
epoch 149, loss 1901.157780
epoch 150, loss 1682.754093
epoch 151, loss 1796.088754
epoch 152, loss 3343.093651
epoch 153, loss 2545.160296
epoch 154, loss 1969.396439
epoch 155, loss 2506.291714
epoch 156, loss 1725.370095
epoch 157, loss 1444.024795
epoch 158, loss 1354.910049
epoch 159, loss 2009

In [30]:
test_pred=model.predict(x_test)

In [35]:
y_test.shape

(127,)

In [44]:
def test(self,x_test):
  import numpy as np
  return np.sqrt((test_pred.detach().numpy()-y_test.reshape(127,1))**2).sum()/127


11.455959119597653

In [ ]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression(normalize=True)
regressor.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)

In [ ]:
from sklearn import metrics
y_pred = regressor.predict(x_train)
print('Mean Squared Error:', metrics.mean_squared_error(y_train, y_pred))

Mean Squared Error: 22.391619489912248


In [ ]:
regressor.coef_


array([[-1.28676633e-01,  5.26505476e-02,  2.80206022e-02,
         1.42212169e+00, -2.39408026e+01,  3.57147848e+00,
         2.10866786e-02, -1.77121559e+00,  3.48418153e-01,
        -1.22253772e-02, -1.06219163e+00,  1.03811454e-02,
        -5.81768637e-01]])

In [ ]:
model.w

tensor([[-0.0287],
        [ 0.0443],
        [ 0.3008],
        [-1.9660],
        [ 0.3276],
        [ 0.1506],
        [ 0.0473],
        [ 0.7430],
        [ 0.2690],
        [-0.0096],
        [ 0.7203],
        [ 0.0127],
        [-0.0624]], dtype=torch.float64, requires_grad=True)